# Preparación de datos
## Importar bibliotecas

In [1]:
import os
import polars as pl
import polars.selectors as cs
import pandas as pd
from datetime import date, time
import numpy as np
import pyarrow.parquet as pq

In [ ]:
# from os import listdir
# from os.path import isfile, join

In [2]:


# Ruta al directorio que contiene los archivos Parquet
directorio = "..\\datasets\\raw\\"

# Obtener la lista de archivos en el directorio
archivos_parquet = [f for f in os.listdir(directorio) if os.path.isfile(os.path.join(directorio, f)) and f.endswith('.parquet')]

# Crear un diccionario para almacenar los DataFrames de Polars
diccionario_dataframes = {}

# Iterar sobre los archivos Parquet
for archivo in archivos_parquet:
    # Verificar si el archivo contiene "yellow", ".parquet" y "2023" en su nombre
    if 'yellow' in archivo and '.parquet' in archivo and '2023' in archivo:
        # Obtener el mes del archivo
        mes = archivo.split('_')[2].split('-')[1].split('.')[0]  # Extraer el mes del nombre del archivo
        
        # Construir la ruta completa al archivo
        ruta_archivo = os.path.join(directorio, archivo)
        
        # Leer el archivo Parquet en un DataFrame de Polars
        df = pl.read_parquet(ruta_archivo)
        
        # Agregar el DataFrame al diccionario utilizando el nombre del archivo como clave
        diccionario_dataframes[mes] = df

# Ahora el diccionario 'diccionario_dataframes' contiene un DataFrame por mes para el año 2023
# de los archivos Parquet que contienen "yellow" en su nombre.


In [3]:
# Iterar sobre el diccionario de dataframes
for nombre, df in diccionario_dataframes.items():
    print(f"Información del DataFrame '{nombre}':")
    print("Cantidad de registros :", len(df), "Cantidad de las Columnas :", len(df.columns))
    print("Schema")
    #print(df.schema)
    print("\n")

Información del DataFrame '01':
Cantidad de registros : 3066766 Cantidad de las Columnas : 19
Schema


Información del DataFrame '02':
Cantidad de registros : 2913955 Cantidad de las Columnas : 19
Schema


Información del DataFrame '03':
Cantidad de registros : 3403766 Cantidad de las Columnas : 19
Schema


Información del DataFrame '04':
Cantidad de registros : 3288250 Cantidad de las Columnas : 19
Schema


Información del DataFrame '05':
Cantidad de registros : 3513649 Cantidad de las Columnas : 19
Schema


Información del DataFrame '06':
Cantidad de registros : 3307234 Cantidad de las Columnas : 19
Schema


Información del DataFrame '07':
Cantidad de registros : 2907108 Cantidad de las Columnas : 19
Schema


Información del DataFrame '08':
Cantidad de registros : 2824209 Cantidad de las Columnas : 19
Schema


Información del DataFrame '09':
Cantidad de registros : 2846722 Cantidad de las Columnas : 19
Schema


Información del DataFrame '10':
Cantidad de registros : 3522285 Cantidad 

In [4]:
# Supongamos que diccionario_dataframes es tu diccionario que contiene los DataFrames
# Reemplaza diccionario_dataframes con el nombre de tu diccionario

# Tomar el primer DataFrame del diccionario
primer_df = next(iter(diccionario_dataframes.values()))

# Definir un nuevo esquema con los tipos de datos y nombres de columna deseados
nuevo_esquema = {
    "VendorID": pl.Int32,
    "tpep_pickup_datetime": pl.Datetime(time_unit='ns', time_zone=None),
    "tpep_dropoff_datetime": pl.Datetime(time_unit='ns', time_zone=None),
    "passenger_count": pl.Int64,
    "trip_distance": pl.Float64,
    "RatecodeID": pl.Int64,
    "store_and_fwd_flag": pl.String,
    "PULocationID": pl.Int32,
    "DOLocationID": pl.Int32,
    "payment_type": pl.Int64,
    "fare_amount": pl.Float64,
    "extra": pl.Float64,
    "mta_tax": pl.Float64,
    "tip_amount": pl.Float64,
    "tolls_amount": pl.Float64,
    "improvement_surcharge": pl.Float64,
    "total_amount": pl.Float64,
    "congestion_surcharge": pl.Float64,
    "airport_fee": pl.Float64,  # He cambiado el nombre de la columna a "airport_fee"
}

# Renombrar las columnas y cambiar los tipos de datos del DataFrame
df = primer_df.select([
    pl.col(col).cast(dtype).alias(col) for col, dtype in nuevo_esquema.items()
])

# Cambiar el nombre de la columna "airport_fee" a "Airport_fee"
df = df.with_columns(primer_df['airport_fee'].alias('Airport_fee'))
df = df.drop('airport_fee')
# Reemplazar el DataFrame original en el diccionario con el DataFrame modificado
nombre_clave = next(iter(diccionario_dataframes))  # Obtener la clave del primer DataFrame
diccionario_dataframes[nombre_clave] = df

# Verificar el resultado
print(diccionario_dataframes[nombre_clave].schema)

OrderedDict([('VendorID', Int32), ('tpep_pickup_datetime', Datetime(time_unit='ns', time_zone=None)), ('tpep_dropoff_datetime', Datetime(time_unit='ns', time_zone=None)), ('passenger_count', Int64), ('trip_distance', Float64), ('RatecodeID', Int64), ('store_and_fwd_flag', String), ('PULocationID', Int32), ('DOLocationID', Int32), ('payment_type', Int64), ('fare_amount', Float64), ('extra', Float64), ('mta_tax', Float64), ('tip_amount', Float64), ('tolls_amount', Float64), ('improvement_surcharge', Float64), ('total_amount', Float64), ('congestion_surcharge', Float64), ('Airport_fee', Float64)])


In [5]:
# Iterar sobre el diccionario de dataframes
# for nombre, df in diccionario_dataframes.items():
#     print(f"Información del DataFrame '{nombre}':")
#     print("Cantidad de registros :", len(df), "Cantidad de las Columnas :", len(df.columns))
#     print("Schema")
#     print(df.schema)
#     print("\n")

In [5]:
# Ahora que todos los DataFrames tienen las mismas columnas, puedes proceder a concatenarlos.
dataframes = list(diccionario_dataframes.values())

# Concatenar verticalmente todos los DataFrames
df_concatenado = pl.concat(dataframes)


In [6]:
df_concatenado.shape

(38310226, 19)

In [7]:
df_concatenado.schema

OrderedDict([('VendorID', Int32),
             ('tpep_pickup_datetime',
              Datetime(time_unit='ns', time_zone=None)),
             ('tpep_dropoff_datetime',
              Datetime(time_unit='ns', time_zone=None)),
             ('passenger_count', Int64),
             ('trip_distance', Float64),
             ('RatecodeID', Int64),
             ('store_and_fwd_flag', String),
             ('PULocationID', Int32),
             ('DOLocationID', Int32),
             ('payment_type', Int64),
             ('fare_amount', Float64),
             ('extra', Float64),
             ('mta_tax', Float64),
             ('tip_amount', Float64),
             ('tolls_amount', Float64),
             ('improvement_surcharge', Float64),
             ('total_amount', Float64),
             ('congestion_surcharge', Float64),
             ('Airport_fee', Float64)])

In [8]:
df_concatenado.head()

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
i32,datetime[ns],datetime[ns],i64,f64,i64,str,i32,i32,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2,2023-01-01 00:32:10,2023-01-01 00:40:36,1,0.97,1,"""N""",161,141,2,9.3,1.0,0.5,0.0,0.0,1.0,14.3,2.5,0.0
2,2023-01-01 00:55:08,2023-01-01 01:01:27,1,1.1,1,"""N""",43,237,1,7.9,1.0,0.5,4.0,0.0,1.0,16.9,2.5,0.0
2,2023-01-01 00:25:04,2023-01-01 00:37:49,1,2.51,1,"""N""",48,238,1,14.9,1.0,0.5,15.0,0.0,1.0,34.9,2.5,0.0
1,2023-01-01 00:03:48,2023-01-01 00:13:25,0,1.9,1,"""N""",138,7,1,12.1,7.25,0.5,0.0,0.0,1.0,20.85,0.0,1.25
2,2023-01-01 00:10:29,2023-01-01 00:21:19,1,1.43,1,"""N""",107,79,1,11.4,1.0,0.5,3.28,0.0,1.0,19.68,2.5,0.0


# tpep_pickup_datetime|tpep_dropoff_datetime / fechas  a segundos y fechas unicamente
# trip_distance tratar las filas con distanca cero 
# RatecodeID| 6 datos adminitidos  (99) valores faltantes
# payment_type 1 y 2  

In [9]:
df_nyc = df_concatenado.drop_nulls()

In [10]:
df_nyc.shape

(37000870, 19)

## General Preprocessing
### Preprocesamiento Variables `Temporales` 

In [11]:
df_nyc.head()

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
i32,datetime[ns],datetime[ns],i64,f64,i64,str,i32,i32,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2,2023-01-01 00:32:10,2023-01-01 00:40:36,1,0.97,1,"""N""",161,141,2,9.3,1.0,0.5,0.0,0.0,1.0,14.3,2.5,0.0
2,2023-01-01 00:55:08,2023-01-01 01:01:27,1,1.1,1,"""N""",43,237,1,7.9,1.0,0.5,4.0,0.0,1.0,16.9,2.5,0.0
2,2023-01-01 00:25:04,2023-01-01 00:37:49,1,2.51,1,"""N""",48,238,1,14.9,1.0,0.5,15.0,0.0,1.0,34.9,2.5,0.0
1,2023-01-01 00:03:48,2023-01-01 00:13:25,0,1.9,1,"""N""",138,7,1,12.1,7.25,0.5,0.0,0.0,1.0,20.85,0.0,1.25
2,2023-01-01 00:10:29,2023-01-01 00:21:19,1,1.43,1,"""N""",107,79,1,11.4,1.0,0.5,3.28,0.0,1.0,19.68,2.5,0.0


In [12]:
from datetime import datetime

In [13]:
# Convertir las columnas de fechas a tipo DateTime en Polars
df_nyc = df_nyc.with_columns(
    pl.col('tpep_pickup_datetime').dt.date().alias('tpep_pickup_date'),
    pl.col('tpep_dropoff_datetime').dt.date().alias('tpep_dropoff_date')
)

# Calcular la duración del viaje en segundos
df_nyc = df_nyc.with_columns(
    (pl.col('tpep_dropoff_datetime') - pl.col('tpep_pickup_datetime')).dt.total_seconds().alias('viaje_segundos')
)
# Eliminar las columnas originales de fecha y hora
df_nyc = df_nyc.drop(['tpep_pickup_datetime', 'tpep_dropoff_datetime'])


In [14]:
df_nyc.head()

VendorID,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,tpep_pickup_date,tpep_dropoff_date,viaje_segundos
i32,i64,f64,i64,str,i32,i32,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,date,date,i64
2,1,0.97,1,"""N""",161,141,2,9.3,1.0,0.5,0.0,0.0,1.0,14.3,2.5,0.0,2023-01-01,2023-01-01,506
2,1,1.1,1,"""N""",43,237,1,7.9,1.0,0.5,4.0,0.0,1.0,16.9,2.5,0.0,2023-01-01,2023-01-01,379
2,1,2.51,1,"""N""",48,238,1,14.9,1.0,0.5,15.0,0.0,1.0,34.9,2.5,0.0,2023-01-01,2023-01-01,765
1,0,1.9,1,"""N""",138,7,1,12.1,7.25,0.5,0.0,0.0,1.0,20.85,0.0,1.25,2023-01-01,2023-01-01,577
2,1,1.43,1,"""N""",107,79,1,11.4,1.0,0.5,3.28,0.0,1.0,19.68,2.5,0.0,2023-01-01,2023-01-01,650


In [14]:
df_nyc.shape

(37000870, 20)

In [15]:
# Eliminar las columnas originales de fecha y hora
df_nyc = df_nyc.drop(['store_and_fwd_flag', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'congestion_surcharge', 'airport_fee'])

In [16]:
df_nyc.shape

(37000870, 12)

In [17]:
# Convertir el DataFrame de Polars a un objeto Arrow
arrow_table = df_nyc.to_arrow()

# Guardar el objeto Arrow en un archivo Arrow
#pq.write_table(arrow_table, 'temperature.arrow')
#pq.write_table(arrow_table, '../datasets/processed/df_yellow.arrow')
pq.write_table(arrow_table, '../datasets/processed/df_yellow.arrow', compression='zstd')

In [15]:
#df_nyc.describe()
df_nyc.schema

OrderedDict([('VendorID', Int32),
             ('passenger_count', Int64),
             ('trip_distance', Float64),
             ('RatecodeID', Int64),
             ('store_and_fwd_flag', String),
             ('PULocationID', Int32),
             ('DOLocationID', Int32),
             ('payment_type', Int64),
             ('fare_amount', Float64),
             ('extra', Float64),
             ('mta_tax', Float64),
             ('tip_amount', Float64),
             ('tolls_amount', Float64),
             ('improvement_surcharge', Float64),
             ('total_amount', Float64),
             ('congestion_surcharge', Float64),
             ('Airport_fee', Float64),
             ('tpep_pickup_date', Date),
             ('tpep_dropoff_date', Date),
             ('viaje_segundos', Int64)])

In [16]:
# Calcular la media de la columna 'tviaje_segundos'
mean_seconds = df_nyc['viaje_segundos'].mean()

# Calcular la mediana de la columna 'tviaje_segundos'
median_seconds = df_nyc['viaje_segundos'].median()

# Calcular la desviación estándar de la columna 'tviaje_segundos'
std_seconds = df_nyc['viaje_segundos'].std()

# Obtener el valor mínimo de la columna 'viaje_segundos'
min_seconds = df_nyc['viaje_segundos'].min()

# Obtener el valor máximo de la columna 'viaje_segundos'
max_seconds = df_nyc['viaje_segundos'].max()

# Imprimir los resultados
print("Minimo:", min_seconds)
print("Maximo:", max_seconds)
print("Media:", mean_seconds)
print("Mediana:", median_seconds)
print("Desviación estándar:", std_seconds)


Minimo: -1694897908
Maximo: 601751
Media: 997.5984043618433
Mediana: 752.0
Desviación estándar: 278647.64816650894


In [17]:
# Filtrar los registros con valores negativos en la columna 'viaje_segundos'
negativos = df_nyc.filter(pl.col('viaje_segundos') < 0.0)

# Obtener la cantidad de registros con valores negativos
cantidad_negativos = negativos.height
print("Cantidad de registros con valores negativos en 'viaje_segundos':", cantidad_negativos)



Cantidad de registros con valores negativos en 'viaje_segundos': 819


In [18]:
# Eliminar los registros con valores negativos en la columna 'viaje_segundos'
#df_nyc = df_nyc.drop(pl.col('viaje_segundos') < 0.0)
# Filtrar las filas que cumplen la condición
df_yellow = df_nyc.filter(pl.col('viaje_segundos') >= 0)

: 

In [5]:
#df = pl.scan_ipc("temperature.arrow")
# create a SQL context, registering the frame as a table
#sql = pl.SQLContext(my_table=df)
# create a SQL query to execute
# Leer el archivo Arrow
arrow_table = pq.read_table('../datasets/processed/temperature.arrow')

# Convertir el objeto Arrow a un DataFrame de Polars
df_polars = pl.from_arrow(arrow_table)

In [6]:
df_polars 

event_date,temperature_2m
"datetime[ns, UTC]",f32
2022-01-01 00:00:00 UTC,8.346
2022-01-01 01:00:00 UTC,9.146
2022-01-01 02:00:00 UTC,7.996
2022-01-01 03:00:00 UTC,8.046
2022-01-01 04:00:00 UTC,7.646
…,…
2023-12-31 19:00:00 UTC,6.396
2023-12-31 20:00:00 UTC,6.296
2023-12-31 21:00:00 UTC,5.796


In [8]:
import pandas as pd
import polars as pl
import pyarrow.parquet as pq
import pyarrow.dataset as ds

# Ejemplo de DataFrame
df = pd.DataFrame({
    'A': [1, 2, 3],
    'B': [4, 5, 6]
})

# Convertir el DataFrame de Pandas a un DataFrame de Polars
df_polars = pl.from_pandas(df)

# Convertir el DataFrame de Polars a un objeto Arrow
arrow_table = df_polars.to_arrow()

# Guardar el objeto Arrow en un archivo Arrow
pq.write_table(arrow_table, 'data.arrow')

# Leer el archivo Arrow utilizando pl.scan_pyarrow_dataset()
dset = ds.dataset("data.arrow", format="ipc")  
df_loaded = (
    pl.scan_pyarrow_dataset(dset)
    .to_pandas()  # Convertir a DataFrame de Pandas para mostrarlo
)

# Mostrar el DataFrame cargado
print(df_loaded)



   A  B
0  1  4
1  2  5
2  3  6


In [19]:
import polars as pl
import pyarrow.parquet as pq

# Leer el archivo Arrow
arrow_table = pq.read_table('../datasets/processed/temperature.arrow')

# Convertir el objeto Arrow a un DataFrame de Polars
df_polars = pl.from_arrow(arrow_table)




In [28]:
df_polars

event_date,temperature_2m
"datetime[ns, UTC]",f32
2022-01-01 00:00:00 UTC,8.346
2022-01-01 01:00:00 UTC,9.146
2022-01-01 02:00:00 UTC,7.996
2022-01-01 03:00:00 UTC,8.046
2022-01-01 04:00:00 UTC,7.646
…,…
2023-12-31 19:00:00 UTC,6.396
2023-12-31 20:00:00 UTC,6.296
2023-12-31 21:00:00 UTC,5.796


In [38]:
# Crear un contexto SQL y registrar el DataFrame como una tabla
sql = pl.SQLContext(frames={"df_polars": df_polars})

result = sql.execute(
    "SELECT * FROM df_polars WHERE temperature_2m < 10"
)



# Convertir el resultado a un DataFrame de Pandas para mostrarlo
result_df = result.collect().to_pandas()

# Mostrar el DataFrame resultante
print(result_df)


                    event_date  temperature_2m
0    2022-01-01 00:00:00+00:00           8.346
1    2022-01-01 01:00:00+00:00           9.146
2    2022-01-01 02:00:00+00:00           7.996
3    2022-01-01 03:00:00+00:00           8.046
4    2022-01-01 04:00:00+00:00           7.646
...                        ...             ...
7169 2023-12-31 19:00:00+00:00           6.396
7170 2023-12-31 20:00:00+00:00           6.296
7171 2023-12-31 21:00:00+00:00           5.796
7172 2023-12-31 22:00:00+00:00           4.396
7173 2023-12-31 23:00:00+00:00           3.446

[7174 rows x 2 columns]


In [25]:
type(result_df)

pandas.core.frame.DataFrame